In [1]:
import requests, time, random
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date
from selenium.webdriver.common.keys import Keys
#!pip install psycopg2

In [2]:
#fetching data from database
# note that we are fetching the first 100 results to be scrapped, this parameter can be changed in the sql query

import psycopg2
import pandas.io.sql as psql

try:
    connection= psycopg2.connect(user="postgres",password="root",host="127.0.0.1", port="5432",database="Biat_Client_Scorer_DataWarehouse")
    
    postgre_Query = 'select * from "DATA_CUSTOMER" where "EMAIL" IS NOT NULL LIMIT 100 ;'

    df = psql.read_sql(postgre_Query, connection)
    if(connection):
        connection.close()
        print("PostgreSQL connection is closed")

except (Exception, psycopg2.Error) as error :
    print ("Error while fetching data from PostgreSQL", error)



PostgreSQL connection is closed


In [3]:
df

,CODE_CLIENT,NAME_LASTNAME,DESCRIPTION,LOCATION,CONNECTIONS,JOB_TITLE,COMPANY_NAME,JOB_DURATION,COLLEGES,DEGREES,CERTIFICATIONS,FLAGGED,TIMESTAMP,EMAIL,DURATION,PROFILE_LINK
0,123789,None,None,None,None,None,None,None,None,None,None,0,2020-09-25 23:40:38.112,ghadast@gmail.com,None,None
1,123963,None,None,None,None,None,None,None,None,None,None,0,2020-09-25 23:40:38.504,heithem.dridi@yahoo.fr,None,None
2,123741,None,None,None,None,None,None,None,None,None,None,0,2020-09-25 23:40:38.505,ounishaima1993@gmail.com,None,None
3,123852,None,None,None,None,None,None,None,None,None,None,0,2020-09-25 23:40:38.506,marwene.elhechmi@esprit.tn,None,None


In [4]:
#filtering data based on date and flag



df['TIME_DIFF_IN_DAYS']=(pd.to_datetime(date.today()) - (df['TIMESTAMP'])).dt.days
filtered_df=df[(df['FLAGGED']==0)]


#Normally if we hadn't done any scrapping in the last 60 days we do it again
#so we filter based on TIME_DIFF_IN_DAYS
filtered_df=df[((df['TIME_DIFF_IN_DAYS'] > 60) & (df['FLAGGED'] == 1) ) | df['FLAGGED'] == 0]

#We filter the columns
filtered_df=filtered_df[['CODE_CLIENT','EMAIL','TIMESTAMP','TIME_DIFF_IN_DAYS','FLAGGED']] 

filtered_df


,CODE_CLIENT,EMAIL,TIMESTAMP,TIME_DIFF_IN_DAYS,FLAGGED
0,123789,ghadast@gmail.com,2020-09-25 23:40:38.112,-1,0
1,123963,heithem.dridi@yahoo.fr,2020-09-25 23:40:38.504,-1,0
2,123741,ounishaima1993@gmail.com,2020-09-25 23:40:38.505,-1,0
3,123852,marwene.elhechmi@esprit.tn,2020-09-25 23:40:38.506,-1,0


In [5]:
if(len(filtered_df)>0):
    browser = webdriver.Chrome('./chromedriver.exe')

    browser.get('https://login.live.com/login.srf')
    browser.maximize_window()
    file = open('configOutlook.txt')
    lines = file.readlines()
    username = lines[0]
    password = lines[1]



    #we'll get the emails csv here
    emails=filtered_df['EMAIL']


    #feeding user email
    elementID = browser.find_element_by_id('i0116')
    elementID.send_keys(username)




    #feeding password
    elementID = browser.find_element_by_id('i0118')
    elementID.send_keys(password)
    time.sleep(3)
    WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.ID, "idSIButton9"))).click()
    try:
        time.sleep(5)
        WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.ID, "idSIButton9"))).click()
    except:
        pass
    
    


    #Going to /people to search by email
    browser.get('https://outlook.live.com/people')

    time.sleep(7)

    info={'NAME_LASTNAME':[],'DESCRIPTION':[]}

    for i in range(len(emails)):
        if (i==0):
            time.sleep(7)
        newContact=browser.find_elements_by_xpath('//*[@id="app"]/div/div[2]/div[1]/div[1]/div/div[1]/div[2]/div/div/div/div/div/div/div/div/span/button[1]')[0]
        browser.execute_script("arguments[0].click();", newContact)



        WebDriverWait(browser, 40).until(EC.element_to_be_clickable((By.ID, "GivenName"))).click()
        #browser.find_element_by_id('GivenName').send_keys('Client 3')
        browser.find_element_by_id('PersonaEmails1-0').send_keys(emails[i])

        browser.find_elements_by_xpath('//*[@id="lpc-contact-editor"]/section[2]/div[2]/div/button[1]/span')[0].click()
        time.sleep(7)

        #Selecting new Contact and getting linkedIn Name and Last Name and description
        linkedIn=WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.NAME, "LinkedIn")))
        #linkedIn=browser.find_element_by_name('LinkedIn')
        browser.execute_script("arguments[0].click();", linkedIn)

        #saving description into file
        src = browser.page_source
        soup = BeautifulSoup(src, "html.parser")
        description_object=soup.find('div',{'data-log-name':'Title'})
        description=soup.find('div',{'data-log-name':'Title'}).get_text().strip()
        name_lastName=description_object.find_previous_sibling().get_text().strip()

        info['NAME_LASTNAME'].append(name_lastName)
        info['DESCRIPTION'].append(description)



    print('done')   


done


# Note that you can delete the outlook contacts created manually after finishing the scrapping to keep the contact tab clean from redundant contacts 

In [6]:

info_df=pd.DataFrame(info,columns=['NAME_LASTNAME','DESCRIPTION'])
info_df   


,NAME_LASTNAME,DESCRIPTION
0,Ghada Saguer Trabelsi,IT Analyst Data chez BIAT
1,Heithem Dridi,Analyste Developpeur BI BigData chez BIAT\nTal...
2,OUNI Shaima,Ingénieur JEE/Angular chez Banque Internation...
3,Marwene Elhechmi,Ingénieur Full Stack Python/Java/JavaEE/Spring...


In [7]:
filtered_df

,CODE_CLIENT,EMAIL,TIMESTAMP,TIME_DIFF_IN_DAYS,FLAGGED
0,123789,ghadast@gmail.com,2020-09-25 23:40:38.112,-1,0
1,123963,heithem.dridi@yahoo.fr,2020-09-25 23:40:38.504,-1,0
2,123741,ounishaima1993@gmail.com,2020-09-25 23:40:38.505,-1,0
3,123852,marwene.elhechmi@esprit.tn,2020-09-25 23:40:38.506,-1,0


In [8]:

joined_df=filtered_df.join(info_df)
joined_df.drop('TIME_DIFF_IN_DAYS',axis=1,inplace=True) 
joined_df

,CODE_CLIENT,EMAIL,TIMESTAMP,FLAGGED,NAME_LASTNAME,DESCRIPTION
0,123789,ghadast@gmail.com,2020-09-25 23:40:38.112,0,Ghada Saguer Trabelsi,IT Analyst Data chez BIAT
1,123963,heithem.dridi@yahoo.fr,2020-09-25 23:40:38.504,0,Heithem Dridi,Analyste Developpeur BI BigData chez BIAT\nTal...
2,123741,ounishaima1993@gmail.com,2020-09-25 23:40:38.505,0,OUNI Shaima,Ingénieur JEE/Angular chez Banque Internation...
3,123852,marwene.elhechmi@esprit.tn,2020-09-25 23:40:38.506,0,Marwene Elhechmi,Ingénieur Full Stack Python/Java/JavaEE/Spring...


# We have completed name, lastname and description scrapping based on email from outlook
# We will now scrap directly from linkedIN

In [12]:
if(len(filtered_df)>0):
    #Opening and logging into LinkedIN
    browser = webdriver.Chrome('./chromedriver.exe')
    browser.maximize_window()
    browser.get('https://www.linkedin.com/uas/login')
    file = open('config.txt')
    lines = file.readlines()
    username = lines[0]
    password = lines[1]
    elementID = browser.find_element_by_id('username')
    elementID.send_keys(username)
    elementID = browser.find_element_by_id('password')
    elementID.send_keys(password)
    elementID.submit()

    linkedIn_columns=['LOCATION','CONNECTIONS','JOB_TITLE','COMPANY_NAME','DURATION','COLLEGES','DEGREES','CERTIFICATIONS','PROFILE_LINK']
    linkedIn_info={'LOCATION':[],'CONNECTIONS':[],'JOB_TITLE':[],'COMPANY_NAME':[],'DURATION':[],'COLLEGES':[],'DEGREES':[],'CERTIFICATIONS':[],'PROFILE_LINK':[]}
    for i in range(len(joined_df)):
        #searching for profile with name lastname and description
        search=WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ember16"]/input')))
        search.send_keys(joined_df['NAME_LASTNAME'][i]+' '+joined_df['DESCRIPTION'][i].split(' ')[0]+' '+ joined_df['DESCRIPTION'][i].split(' ')[1])
        search.send_keys(Keys.ENTER)



        profile=WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, joined_df['NAME_LASTNAME'][i])))
        browser.execute_script("arguments[0].click();", profile)


                # Get scroll height
        last_height = browser.execute_script("return document.body.scrollHeight")

        for j in range(3):
            # Scroll down to bottom
            browser.execute_script("window.scrollTo(0, 2*document.body.scrollHeight/3);")
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight/3);")

            # Wait to load page
            time.sleep(5)

            # Calculate new scroll height and compare with last scroll height
            new_height = browser.execute_script("return document.body.scrollHeight")
            if last_height==new_height:
                break
            last_height = new_height


        src = browser.page_source
        profile_link=browser.current_url
        soup = BeautifulSoup(src, "html.parser")


        name_div = soup.find('div', {'class': 'flex-1 mr5'})
        name_loc = name_div.find_all('ul')
        try:
            location = name_loc[1].find('li').get_text().strip()
        except:
            location = 'none'

        try:
            connections = name_loc[1].find_all('li')
            connections = connections[1].get_text().strip().split(' ')[0]
        except:
            connections= 'none'

        exp_section = soup.find('section', {'id': 'experience-section'})
        exp_section = exp_section.find('ul')
        li_tags = exp_section.find('div')
        a_tags = li_tags.find('a')

        try:
            job_title = a_tags.find('h3').get_text().strip()
        except:
            job_title='none'

        try:
            company_name=a_tags.find_all('p')[1].get_text().strip()
        except:
            company_name='none'

        try:
            exp=a_tags.find_all('span')[3].get_text().strip()
        except:
            exp='none'


        try:
            edu_section = soup.find('section', {'id': 'education-section'})
            college_names=edu_section.find_all('h3',{'class':'pv-entity__school-name'})
            colleges=''
            for college in college_names:

                colleges+=college.get_text().strip()+', '
            last_char_index = colleges.rfind(",")
            colleges = colleges[:last_char_index] + colleges[last_char_index+1:]
        except:
            colleges='none'

        try:
            degree_names=edu_section.find_all('span',{'class':'pv-entity__comma-item'})
            degrees=''
            for degree in degree_names:
                degrees+=degree.get_text().strip()+' ,'
            last_char_index = degrees.rfind(",")
            degrees= degrees[:last_char_index] + degrees[last_char_index+1:]
        except:
            degrees='none'

        certification_section =soup.find('section',{'id':'certifications-section'})
        try:
            certification_names=certification_section.find_all('h3',{'class':'t-16 t-bold'})
            certifications=''
            for certification in certification_names:
                certifications+=certification.get_text().strip()+', '
            last_char_index = certifications.rfind(",")
            certifications= certifications[:last_char_index] + certifications[last_char_index+1:]
        except:
            certifications='none'
        linkedIn_info['PROFILE_LINK'].append(profile_link)
        linkedIn_info['LOCATION'].append(location)
        linkedIn_info['CONNECTIONS'].append(connections)
        linkedIn_info['JOB_TITLE'].append(job_title)
        linkedIn_info['COMPANY_NAME'].append(company_name)
        linkedIn_info['DURATION'].append(exp)
        linkedIn_info['COLLEGES'].append(colleges)
        linkedIn_info['DEGREES'].append(degrees)
        linkedIn_info['CERTIFICATIONS'].append(certifications)

        #setting flag to 1 and updating timestamp
        joined_df.at[i,'FLAGGED']=1
        joined_df.at[i,'TIMESTAMP']=datetime.now()



    print('done')

done


In [13]:
info_df=pd.DataFrame(linkedIn_info,columns=linkedIn_columns)
info_df

,LOCATION,CONNECTIONS,JOB_TITLE,COMPANY_NAME,DURATION,COLLEGES,DEGREES,CERTIFICATIONS,PROFILE_LINK
0,"Beni Khalled, Nabeul Governorate, Tunisia",198,IT Analyst Data,BIAT,3 yrs,"Xpertos, Ecole Nationale d’Ingénieurs de Carth...","""La méthode Agile- Scrum"" ,Gestion de projets ...","TOIEC, CCNA 1",https://www.linkedin.com/in/ghada-saguer-trabe...
1,"Bou Mhel El Bassatine, Ben Arous Governorate, ...",195,Analyste developpeur BI,BIAT,2 yrs,Faculté des Sciences Economiques et de Gestion...,"Master's degree ,business intelligence ,Méthod...",none,https://www.linkedin.com/in/heithem-dridi-0874...
2,"Tunis, Tunisia",500+,Ingénieur JEE/Angular,BIAT,1 yr 11 mos,"KEYRUS, Xpertos management, Institut Supèrieur...","Formation QLICKVIEW ,Formation "" la méthode ag...",none,https://www.linkedin.com/in/ounishaima/
3,"Tunis Governorate, Tunisia",500+,Ingénieur Full Stack Python/Java/JavaEE/Spring...,BIAT,2 yrs 5 mos,"Université Nice Sophia Antipolis, BIAT : FORMA...","Master's degree , Informatique Finance ,Mathém...",none,https://www.linkedin.com/in/marwene-elhechmi-0...


In [14]:
final_df=joined_df.join(info_df)
final_df



,CODE_CLIENT,EMAIL,TIMESTAMP,FLAGGED,NAME_LASTNAME,DESCRIPTION,LOCATION,CONNECTIONS,JOB_TITLE,COMPANY_NAME,DURATION,COLLEGES,DEGREES,CERTIFICATIONS,PROFILE_LINK
0,123789,ghadast@gmail.com,2020-09-25 23:50:10.849634,1,Ghada Saguer Trabelsi,IT Analyst Data chez BIAT,"Beni Khalled, Nabeul Governorate, Tunisia",198,IT Analyst Data,BIAT,3 yrs,"Xpertos, Ecole Nationale d’Ingénieurs de Carth...","""La méthode Agile- Scrum"" ,Gestion de projets ...","TOIEC, CCNA 1",https://www.linkedin.com/in/ghada-saguer-trabe...
1,123963,heithem.dridi@yahoo.fr,2020-09-25 23:50:27.774644,1,Heithem Dridi,Analyste Developpeur BI BigData chez BIAT\nTal...,"Bou Mhel El Bassatine, Ben Arous Governorate, ...",195,Analyste developpeur BI,BIAT,2 yrs,Faculté des Sciences Economiques et de Gestion...,"Master's degree ,business intelligence ,Méthod...",none,https://www.linkedin.com/in/heithem-dridi-0874...
2,123741,ounishaima1993@gmail.com,2020-09-25 23:50:44.464534,1,OUNI Shaima,Ingénieur JEE/Angular chez Banque Internation...,"Tunis, Tunisia",500+,Ingénieur JEE/Angular,BIAT,1 yr 11 mos,"KEYRUS, Xpertos management, Institut Supèrieur...","Formation QLICKVIEW ,Formation "" la méthode ag...",none,https://www.linkedin.com/in/ounishaima/
3,123852,marwene.elhechmi@esprit.tn,2020-09-25 23:51:01.108761,1,Marwene Elhechmi,Ingénieur Full Stack Python/Java/JavaEE/Spring...,"Tunis Governorate, Tunisia",500+,Ingénieur Full Stack Python/Java/JavaEE/Spring...,BIAT,2 yrs 5 mos,"Université Nice Sophia Antipolis, BIAT : FORMA...","Master's degree , Informatique Finance ,Mathém...",none,https://www.linkedin.com/in/marwene-elhechmi-0...


# After Scrapping the necessary data we need to update the values in PostgreSQL database

In [15]:
#Connecting to database
try:
    connection= psycopg2.connect(user="postgres",password="root",host="127.0.0.1", port="5432",database="Biat_Client_Scorer_DataWarehouse")
    cursor=connection.cursor()

    for k in range(len(final_df)):

        postgre_update_query = 'UPDATE "DATA_CUSTOMER" SET "TIMESTAMP" = %s , "FLAGGED" = %s , "NAME_LASTNAME" = %s , "DESCRIPTION" = %s , "LOCATION" = %s  , "CONNECTIONS" = %s , "JOB_TITLE" = %s , "COMPANY_NAME" = %s , "DURATION" = %s , "COLLEGES" = %s , "DEGREES" = %s , "CERTIFICATIONS" = %s , "PROFILE_LINK" = %s   where "CODE_CLIENT" = %s;'
        cursor.execute(postgre_update_query,(final_df['TIMESTAMP'][k],final_df['FLAGGED'][k].item(),final_df['NAME_LASTNAME'][k],final_df['DESCRIPTION'][k],final_df['LOCATION'][k],final_df['CONNECTIONS'][k],final_df['JOB_TITLE'][k],final_df['COMPANY_NAME'][k],final_df['DURATION'][k],final_df['COLLEGES'][k],final_df['DEGREES'][k],final_df['CERTIFICATIONS'][k],final_df['PROFILE_LINK'][k],final_df['CODE_CLIENT'][k].item()))
        connection.commit()

    if(connection):
        connection.close()
        print("PostgreSQL connection is closed")

except (Exception, psycopg2.Error) as error :
    print ("Error while fetching data from PostgreSQL", error)

PostgreSQL connection is closed
